# <center> __M1 Info, Algorithmes et traitement de données__ </center>
# <center> __T.P. Page Rank__ </center>
# <center> *Calvin Massonnet* </center>

In [82]:
#@title Formats des données

# http://ama.liglab.fr/~gaussier/Courses/MastereBigData/TP-pagerank/graphe-samples/tiny-graph.txt
# http://ama.liglab.fr/~gaussier/Courses/MastereBigData/TP-pagerank/graphe-samples/small-20-pages.txt
# http://ama.liglab.fr/~gaussier/Courses/MastereBigData/TP-pagerank/graphe-samples/medium-graph.txt
# http://ama.liglab.fr/~gaussier/Courses/MastereBigData/TP-pagerank/graphe-samples/large-graph.txt

In [83]:
#@markdown - Links 4
Links_4 = [
  (0, 1, 2),
  (1, 0, 2, 3),
  (2, 0, 1),
  (3, 2)
]

In [84]:
#@markdown - Links 6
Links_6 = [
  (0, 2, 3),
  (1, 0, 3, 5),
  (2, 1, 3, 4, 5),
  (3, 0),
  (4, 3),
  (5, 2, 3)
]

In [85]:
#@markdown - Links 20
Links_20 = [
  (0, 19, 4, 2, 16, 5, 9, 11, 7, 10),
  (1, 10, 13, 19, 0, 7, 9, 8),
  (2, 13, 8, 15, 16, 4, 18, 14),
  (3, 6, 15, 0, 11, 7, 1, 17, 5, 19, 9),
  (4, 10, 11, 16, 12, 19, 17, 0, 3, 13, 7),
  (5, 14, 16, 4, 17, 0, 13, 12, 7),
  (6, 3, 15),
  (7, 11, 12, 2, 10, 13, 14, 4),
  (8, 1, 0, 17, 12, 6, 7, 19),
  (9, 11),
  (10, 12, 6, 1, 0, 11, 13),
  (11, 10, 19),
  (12, 5, 17, 19, 15, 8, 2, 4, 9, 18),
  (13, 18, 0, 8),
  (14, 17),
  (15, 17, 13, 16, 6, 10, 0, 4, 2, 7),
  (16, 4, 13, 18, 11, 7, 0, 6, 5, 9),
  (17, 8, 10, 2, 3, 4, 9),
  (18, 3, 1, 7),
  (19, 1, 8, 3, 12, 9, 13, 17, 11)
]

In [86]:
#@markdown - Links 20 In 0
Links_20_In_0 = [
  (0, 19, 4, 2, 16, 5, 9, 11, 7, 10),
  (1, 0, 10, 13, 19, 7, 9, 8),
  (2, 0, 13, 8, 15, 16, 4, 18, 14),
  (3, 0, 6, 15, 11, 7, 1, 17, 5, 19, 9),
  (4, 0, 10, 11, 16, 12, 19, 17, 3, 13, 7),
  (5, 0, 14, 16, 4, 17, 13, 12, 7),
  (6, 0, 3, 15),
  (7, 0, 11, 12, 2, 10, 13, 14, 4),
  (8, 0, 1, 17, 12, 6, 7, 19),
  (9, 0, 11),
  (10, 0, 12, 6, 1, 11, 13),
  (11, 0, 10, 19),
  (12, 0, 5, 17, 19, 15, 8, 2, 4, 9, 18),
  (13, 0, 18, 8),
  (14, 0, 17),
  (15, 0, 17, 13, 16, 6, 10, 4, 2, 7),
  (16, 0, 4, 13, 18, 11, 7, 6, 5, 9),
  (17, 0, 8, 10, 2, 3, 4, 9),
  (18, 0, 3, 1, 7),
  (19, 0, 1, 8, 3, 12, 9, 13, 17, 11)
]

In [87]:
#@markdown - Links 20 Out 0
Links_20_Out_0 = [
  (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19),
  (1, 10, 13, 19),
  (2, 13, 8, 15, 16),
  (3, 6, 15, 11, 7, 1),
  (4, 10, 11, 16, 12, 19),
  (5, 14, 16, 4, 17),
  (6, 3),
  (7, 11, 12, 2),
  (8, 1, 17),
  (9, 11),
  (10, 12, 6, 1, 11, 13),
  (11, 10, 19),
  (12, 5, 17, 19, 15, 8, 2),
  (13, 18, 8),
  (14, 17),
  (15, 17, 13, 16, 6, 10),
  (16, 4, 13, 18, 11),
  (17, 8, 10, 2, 3),
  (18, 3, 1, 7),
  (19, 1, 8, 3, 12, 9)
]

In [88]:
#@title Fonctions utilisées dans l'algorithme du Page Rank
import math

#@markdown - linksToBinaryMatrix(L)
def linksToBinaryMatrix(Links):
  N = len(Links)
  A = [[0] * N] * N
  for n in range(N):
    a = [0] * N
    for l in range(1, len(Links[n])):
      a[Links[n][l]] = 1
    A[n] = a
  return A

#@markdown - transitionMatrix(A, f)
def transitionMatrix(A, f):
  N = len(A)
  P = [[[]] * N] * N
  for i in range(N):
    p = [[]] * N
    sum = 0
    for j in range(N):
      sum += A[i][j]
    for j in range(N):
      if sum != 0:
        p[j] = f * A[i][j] / sum + (1 - f) * (1 / N)
      else:
        p[j] = 1 / N
    P[i] = p
  return P

In [89]:
#@title Jeu de tests des différentes fonctions utilisées dans l'algorithme

def testLinksToBinaryMatrix():
  Links = [(0, 1, 2),
           (1, 0, 3, 2),
           (2, 0, 1),
           (3, 2)]
  A = linksToBinaryMatrix(Links)
  expected_A = [[0, 1, 1, 0],
                [1, 0, 1, 1],
                [1, 1, 0, 0],
                [0, 0, 1, 0]]
  assert A == expected_A
  print("linksToBinaryMatrix() - OK")

def testTransitionMatrix():
  A = [[0, 1, 1, 0],
       [1, 0, 1, 1],
       [1, 1, 0, 0],
       [0, 0, 1, 0]]
  f = 1
  P = transitionMatrix(A, f)
  expected_P = [[0,   1/2, 1/2, 0],
                [1/3, 0,   1/3, 1/3],
                [1/2, 1/2, 0,   0],
                [0,   0,   1,   0]]
  assert P == expected_P
  print("transitionMatrix() - OK")

testLinksToBinaryMatrix()
testTransitionMatrix()
print("Le jeu de tests est passé avec succés.")

linksToBinaryMatrix() - OK
transitionMatrix() - OK
Le jeu de tests est passé avec succés.


In [90]:
#@title Algorithme de Page Rank
import numpy as np

# A - matrice d'adjacence du graphe dirigé
# f - facteur d'amortissement
# e - précision pour le critère d'arrêt
def pageRank(A, f, e):
  P = np.array(transitionMatrix(A, f))
  R = np.array([[1 / len(P)] * len(P)])

  l = 0
  d = e + 1
  while d > e:
    R = np.append(R, [np.matmul(R[l], P)], axis=0)
    d = np.linalg.norm(R[l+1] - R[l])
    l += 1
  return R[l]


A_test = [[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]
A_4 = linksToBinaryMatrix(Links_4)
A_6 = linksToBinaryMatrix(Links_6)
A_20 = linksToBinaryMatrix(Links_20)
A_20_In_0 = linksToBinaryMatrix(Links_20_In_0)
A_20_Out_0 = linksToBinaryMatrix(Links_20_Out_0)
e_3 = pow(10, -3)
e_4 = pow(10, -4)
e_5 = pow(10, -5)
e_8 = pow(10, -8)
e_10 = pow(10, -10)
indent = " "

print("Links test")
print(A_test)
print()
print(indent, "Critère d'arrêt :", e_3)
print()
print(indent, indent, pageRank(A_test, 0.85, e_3))
print()
print(indent, "Critère d'arrêt :", e_4)
print()
print(indent, indent, pageRank(A_test, 0.85, e_4))
print()
print(indent, "Critère d'arrêt :", e_5)
print()
print(indent, indent, pageRank(A_test, 0.85, e_5))
print()
print()
print()
print("Links 4")
print(A_4)
print()
print(indent, "Critère d'arrêt :", e_3)
print()
print(indent, indent, pageRank(A_4, 0.85, e_3))
print()
print(indent, "Critère d'arrêt :", e_4)
print()
print(indent, indent, pageRank(A_4, 0.85, e_4))
print()
print(indent, "Critère d'arrêt :", e_5)
print()
print(indent, indent, pageRank(A_4, 0.85, e_5))
print()
print()
print()
print("Links 6")
print(A_6)
print()
print(indent, "Critère d'arrêt :", e_3)
print()
print(indent, indent, pageRank(A_6, 0.85, e_3))
print()
print(indent, "Critère d'arrêt :", e_4)
print()
print(indent, indent, pageRank(A_6, 0.85, e_4))
print()
print(indent, "Critère d'arrêt :", e_5)
print()
print(indent, indent, pageRank(A_6, 0.85, e_5))
print()
print()
print()
print("Links 20")
print(A_20)
print()
print(indent, "Critère d'arrêt :", e_3)
print()
print(indent, indent, pageRank(A_20, 0.85, e_3))
print()
print(indent, "Critère d'arrêt :", e_4)
print()
print(indent, indent, pageRank(A_20, 0.85, e_4))
print()
print(indent, "Critère d'arrêt :", e_5)
print()
print(indent, indent, pageRank(A_20, 0.85, e_5))
print()
print(indent, "Critère d'arrêt :", e_8)
print()
print(indent, indent, pageRank(A_20, 0.85, e_8))
print()
print(indent, "Critère d'arrêt :", e_10)
print()
print(indent, indent, pageRank(A_20, 0.85, e_10))
print()
print()
print()
print("Links 20 In 0")
print(A_20_In_0)
print()
print(indent, "Critère d'arrêt :", e_3)
print()
print(indent, indent, pageRank(A_20_In_0, 0.85, e_3))
print()
print(indent, "Critère d'arrêt :", e_4)
print()
print(indent, indent, pageRank(A_20_In_0, 0.85, e_4))
print()
print(indent, "Critère d'arrêt :", e_5)
print()
print(indent, indent, pageRank(A_20_In_0, 0.85, e_5))
print()
print()
print()
print("Links 20 Out 0")
print(A_20_Out_0)
print()
print(indent, "Critère d'arrêt :", e_3)
print()
print(indent, indent, pageRank(A_20_Out_0, 0.85, e_3))
print()
print(indent, "Critère d'arrêt :", e_4)
print()
print(indent, indent, pageRank(A_20_Out_0, 0.85, e_4))
print()
print(indent, "Critère d'arrêt :", e_5)
print()
print(indent, indent, pageRank(A_20_Out_0, 0.85, e_5))

Links test
[[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]

  Critère d'arrêt : 0.001

    [0.25 0.25 0.25 0.25]

  Critère d'arrêt : 0.0001

    [0.25 0.25 0.25 0.25]

  Critère d'arrêt : 1e-05

    [0.25 0.25 0.25 0.25]



Links 4
[[0, 1, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [0, 0, 1, 0]]

  Critère d'arrêt : 0.001

    [0.26008975 0.28897247 0.3316411  0.11929669]

  Critère d'arrêt : 0.0001

    [0.26024962 0.28896155 0.33142066 0.11936818]

  Critère d'arrêt : 1e-05

    [0.26023139 0.28895944 0.33143752 0.11937165]



Links 6
[[0, 0, 1, 1, 0, 0], [1, 0, 0, 1, 0, 1], [0, 1, 0, 1, 1, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 1, 1, 0, 0]]

  Critère d'arrêt : 0.001

    [0.29873888 0.06477246 0.18760626 0.30094572 0.06477246 0.08316421]

  Critère d'arrêt : 0.0001

    [0.29896493 0.06482334 0.18744513 0.30074964 0.06482334 0.08319363]

  Critère d'arrêt : 1e-05

    [0.298986   0.06482808 0.1874301  0.30073137 0.06482808 0.08319638]



Links 20
[[0, 0, 1, 0, 1, 1, 0

### **Q.** Choisissez différentes valeurs pour le critère d'arrêt. Qu'est-ce que vous observez ?

**A.** Plus le critère d'arrêt est bas, plus l'algorithme se déroulera. Ce qui augmente la marge d'erreur de l'algorithme.

### **Q** Ajoutez quelques *hubs* (pages qui ont beaucoup de liens sortant) et *autorités* (pages qui ont beaucoup de liens entrant). Quelles pages sont classées le plus haut ?

**A.** Les pages avec beaucoup de liens entrant sont classées plus élevé que les autres pages, tandis que les pages avec beaucoup de liens sortant sont classées moins élevé que les autres pages.

### **Q.** Essayez d'accroître le rang de certaines pages. Expliquez votre méthode et validez-là expérimentalement.

**A.** Pour accroître le rang de certaines pages, il suffit de faire pointer un maximum de pages vers celles-ci et de diminuer le rang des pages élevées non-voulues.

### **Q.** Essayez différentes valeurs pour le facteur d'amortissement λ (ici, f). Quel est le comportement de l'algorithme lorsque λ tend vers 0 ?

**A.** Lorsque λ tend vers 0, l'algorithme est plus rapide et le rand de chaque page tend vers 1/N. En d'autres mots, le rend des pages devient de plus en plus équitable, ce qui rend tout aussi probable de tomber sur une page que sur une autre.